In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp
from ortools.math_opt.python import mathopt

#### Reading and Understanding Data

In [2]:
df1 = pd.read_excel('attachment_2.xlsx', sheet_name='Half-hourly data')
df1.rename(columns={'Unnamed: 0':'timestamp',
                    'Market 1 Price [£/MWh]':'m1_price',
                    'Market 2 Price [£/MWh]':'m2_price'}, inplace=True)
df2  = pd.read_excel('attachment_2.xlsx', sheet_name='Daily data')
df2.rename(columns={'Unnamed: 0':'timestamp',
                    'Market 3 Price [£/MWh]':'m3_price'}, inplace=True)
data = df1.merge(df2, on='timestamp', how='left')
data['date'] = data['timestamp'].dt.date

# data = data[data['date'] <= pd.Timestamp('2018-12-31').date()]
display(data.head(2), data.tail(2))

,timestamp,m1_price,m2_price,Transmission System Electricity Demand [MW],Wind Generation [MW],Solar Generation [MW],Coal Generation [MW],Gas Generation [MW],m3_price,date
0,2018-01-01 00:00:00,48.47,52.18,25506,10715.5,0.01520,1520.0,3796.0,48.6,2018-01-01
1,2018-01-01 00:30:00,49.81,51.60,26267,10650.5,0.00851,1640.0,3796.0,NaN,2018-01-01


,timestamp,m1_price,m2_price,Transmission System Electricity Demand [MW],Wind Generation [MW],Solar Generation [MW],Coal Generation [MW],Gas Generation [MW],m3_price,date
52606,2020-12-31 23:00:00,60.63,63.03,29265,5704.0,0.0,0.0,7939.833333,NaN,2020-12-31
52607,2020-12-31 23:30:00,63.81,69.44,28423,5797.0,0.0,0.0,7394.300000,NaN,2020-12-31


In [3]:
data.describe()

,timestamp,m1_price,m2_price,Transmission System Electricity Demand [MW],Wind Generation [MW],Solar Generation [MW],Coal Generation [MW],Gas Generation [MW],m3_price
count,52608,52608.000000,52608.000000,52608.000000,52608.000000,52608.000000,52608.000000,52608.000000,1096.000000
mean,2019-07-02 23:45:00.410583808,44.451889,47.957905,28866.756710,7209.319913,1352.671308,1011.795406,9388.340348,44.260949
min,2018-01-01 00:00:00,-73.110000,-70.510000,13368.000000,401.000000,0.000000,0.000000,-3.000000,10.800000
25%,2018-10-01 23:52:30,33.320000,35.950000,23519.750000,3592.937500,0.000339,0.000000,5015.712500,34.400000
50%,2019-07-02 23:45:00,43.770000,47.090000,28188.000000,6510.083333,9.335000,300.000000,9058.745192,43.800000
75%,2020-04-01 23:37:30,54.760000,58.620000,33443.250000,10479.062500,2180.000000,1278.041667,13627.943750,54.400000
max,2020-12-31 23:30:00,502.670000,514.270000,49595.000000,18493.750000,9680.000000,11686.000000,23642.500000,94.000000
std,NaN,19.440124,20.351915,6768.594918,4231.086561,2101.129585,1728.927780,5419.830267,13.508754


#### Model Development

In [4]:
# create model
model = mathopt.Model()

# variables
max_charge = 2
max_discharge = 2
max_soc = 4
max_cycles = 5000
charging_eff = 0.95
discharging_eff = 0.95

discharge1 = {t: model.add_variable(lb=0, ub=max_discharge, name=f"discharge1_{t}") for t in data.index}
discharge2 = {t: model.add_variable(lb=0, ub=max_discharge, name=f"discharge2_{t}") for t in data.index}
charge1 = {t: model.add_variable(lb=0, ub=max_charge, name=f"charge1_{t}") for t in data.index}
charge2 = {t: model.add_variable(lb=0, ub=max_charge, name=f"charge2_{t}") for t in data.index}
is_charging = {t: model.add_variable(lb=0, ub=1, is_integer=True, name=f"is_charging_{t}") for t in data.index}
soc = {t: model.add_variable(lb=0, ub=max_soc, name=f"soc_{t}") for t in data.index}

days = data["date"].unique()
discharge3 = {d: model.add_variable(lb=0, ub=max_discharge, name=f"discharge3_{d}") for d in days}
charge3 = {d: model.add_variable(lb=0, ub=max_charge, name=f"charge3_{d}") for d in days}

In [5]:
# objective function
revenue = sum(
    (discharge1[t] * data.loc[t, "m1_price"] + discharge2[t] * data.loc[t, "m2_price"]) * 0.5 for t in data.index
    ) + sum(discharge3[d] * data[data["date"] == d]["m3_price"].iloc[0] * 24 for d in days)
cost = sum(
    (charge1[t] * data.loc[t, "m1_price"] + charge2[t] * data.loc[t, "m2_price"]) * 0.5 for t in data.index
    ) + sum(charge3[d] * data[data["date"] == d]["m3_price"].iloc[0] * 24 for d in days)

model.maximize(revenue - cost)

In [6]:
# constraints
for t in data.index:
    d = data.loc[t, "date"]
    
    # max charging and discharging rate constraint, either charging/discharging constraint
    model.add_linear_constraint(discharge1[t] + discharge2[t] + discharge3[d] <= 2 * (1 - is_charging[t]))
    model.add_linear_constraint(charge1[t] + charge2[t] + charge3[d] <= 2 * is_charging[t])

    # battery soc constraints
    if t == 0:
        model.add_linear_constraint(
            soc[t] == 0 + 0.5 * (charge1[t] + charge2[t] + charge3[d]) * charging_eff - 0.5 * (discharge1[t] + discharge2[t] + discharge3[d]) / discharging_eff
        )
    else:
        model.add_linear_constraint(
            soc[t] == soc[t - 1] + 0.5 * (charge1[t] + charge2[t] + charge3[d]) * charging_eff - 0.5 * (discharge1[t] + discharge2[t] + discharge3[d]) / discharging_eff
        )
# mac cycles constraint
model.add_linear_constraint(sum(charge1[t] + charge2[t] + charge3[data.loc[t, 'date']] for t in data.index) * 0.5 / max_soc <= max_cycles)

<LinearConstraint id: 157824, name: ''>

In [7]:
# solve
params = mathopt.SolveParameters(enable_output=True, relative_gap_tolerance=0.02, random_seed=0)
solver_result = mathopt.solve(model, mathopt.SolverType.HIGHS, params=params)

Coefficient ranges:
  Matrix [1e-01, 6e+00]
  Cost   [5e-03, 2e+03]
  Bound  [1e+00, 4e+00]
  RHS    [2e+00, 5e+03]
Presolving model
157825 rows, 317840 cols, 948039 nonzeros  0s
157825 rows, 212624 cols, 684999 nonzeros  0s
157825 rows, 212624 cols, 684999 nonzeros  0s

Solving MIP model with:
   157825 rows
   212624 cols (52608 binary, 0 integer, 0 implied int., 160016 continuous)
   684999 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   4889178.1       -inf          

In [8]:
if solver_result.termination.reason == mathopt.TerminationReason.OPTIMAL:
    print("Optimal solution found")
    print(f"Objective Function: {solver_result.objective_value():.2f}")
else:
    print("Optimal solution not found")

Optimal solution found
Objective Function: 229459.64


#### Model Results

In [9]:
results = []
for t in data.index:
    d = data.loc[t, "date"]
    results.append({
        "timestamp": data.loc[t, "timestamp"],
        "is_charging": solver_result.variable_values()[is_charging[t]],
        "discharge1": solver_result.variable_values()[discharge1[t]],
        "discharge2": solver_result.variable_values()[discharge2[t]],
        "discharge3": solver_result.variable_values()[discharge3[d]],
        "charge1": solver_result.variable_values()[charge1[t]],
        "charge2": solver_result.variable_values()[charge2[t]],
        "charge3": solver_result.variable_values()[charge3[d]],
        "soc": solver_result.variable_values()[soc[t]],
    })

# Create DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.merge(data[['timestamp', "m1_price", "m2_price", "m3_price"]], on='timestamp', how='left')
results_df.loc[:, results_df.columns != 'timestamp'] = results_df.loc[:, results_df.columns != 'timestamp'].round(2)
results_df["m3_price"] = results_df["m3_price"].ffill()

results_df.head(2)

,timestamp,is_charging,discharge1,discharge2,discharge3,charge1,charge2,charge3,soc,m1_price,m2_price,m3_price
0,2018-01-01 00:00:00,1.0,0.0,0.0,0.0,2.00,0.0,0.0,0.95,48.47,52.18,48.6
1,2018-01-01 00:30:00,1.0,0.0,0.0,0.0,0.22,0.0,0.0,1.05,49.81,51.60,48.6


In [10]:
cycles = sum(results_df[['charge1','charge2','charge3']].sum()) * 0.5 / max_soc // 1
print(f"Total number of cycles: {cycles}")

Total number of cycles: 4334.0


In [11]:
results_df.to_csv('results.csv', index=False)